In [30]:
import os
import pyspark
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
datasets_path = os.path.join('..','movie_recommendation_system', 'datasets')
complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
tags_file = os.path.join(datasets_path, 'ml-latest', 'tags.csv')
links_file = os.path.join(datasets_path, 'ml-latest','links.csv')
movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')

In [31]:
#spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName('my-cool-app') \
    .getOrCreate()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:43951)
Traceback (most recent call last):
  File "/home/yiyeon/spark-3.0.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yiyeon/spark-3.0.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:43951)

In [21]:
#Schema design which are as same as csv
ratingschema = StructType()\
    .add("userId", IntegerType(), True)\
    .add("movieId", IntegerType(), True)\
    .add("rating", DoubleType(), True)\
    .add("timeStamp", IntegerType(), True)
tagsschema = StructType()\
    .add("userId", IntegerType(), True)\
    .add("movieId", IntegerType(), True)\
    .add("tag", StringType(), True)\
    .add("timeStamp", IntegerType(), True)
linkschema = StructType()\
    .add("movieId", IntegerType(), True)\
    .add("IMDBID", IntegerType(), True)\
    .add("TMDBID", IntegerType(), True)

In [22]:
ratingdf = spark.read.format("csv")\
    .option("header",True)\
    .schema(ratingschema)\
    .load(complete_ratings_file)
tagdf = spark.read.format("csv")\
    .option("header",True)\
    .schema(tagsschema)\
    .load(tags_file)
linkdf = spark.read.format("csv")\
    .option("header",True)\
    .schema(linkschema)\
    .load(links_file)

In [15]:
movie = spark.read.option("inferSchema", "true")\
    .option("header", "true").csv(movies_file)
movie

DataFrame[movieId: int, title: string, genres: string]

In [16]:
moviedf = movie.drop("genres")
genredf = movie.drop("title")
userdf = ratingdf.drop("movieId", "rating", "timeStamp")
userdf = userdf.select('userId').distinct()

In [17]:
moviedf, genredf, userdf

(DataFrame[movieId: int, title: string],
 DataFrame[movieId: int, genres: string],
 DataFrame[userId: int])

we need to connect Redis here

# ALS: collaborative filtering for explicit data

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [26]:
spark.driver.memory 15g

SyntaxError: invalid syntax (<ipython-input-26-007b2cf79cdd>, line 1)

In [23]:
ratingforals = ratingdf.drop("timeStamp")
(training, test) = ratingforals.randomSplit([0.8, 0.2])
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

Py4JJavaError: An error occurred while calling o364.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 24, 192.168.1.73, executor driver): java.lang.OutOfMemoryError: Java heap space
	at scala.collection.mutable.ArrayBuilder$ofInt.mkArray(ArrayBuilder.scala:327)
	at scala.collection.mutable.ArrayBuilder$ofInt.resize(ArrayBuilder.scala:333)
	at scala.collection.mutable.ArrayBuilder$ofInt.ensureSize(ArrayBuilder.scala:345)
	at scala.collection.mutable.ArrayBuilder$ofInt.$plus$eq(ArrayBuilder.scala:350)
	at scala.collection.mutable.ArrayBuilder$ofInt.$plus$eq(ArrayBuilder.scala:320)
	at scala.collection.generic.Growable.$anonfun$$plus$plus$eq$1(Growable.scala:62)
	at scala.collection.generic.Growable$$Lambda$8/103887628.apply(Unknown Source)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedS